# Big-4 leagues teams performance analysis

## Description
In this assignment we will read a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). It is important to keep in mind that all the analysis made is from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data we've been given

For each sport we are going to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. 



### Importing and cleaning cities data from wikipedia html
In this section we imported the cities dataframe form wikipedia html, we reduced to only 6 relevant columns, deleted the totals row, and cleaned the columns names.
Additionally we cleaned the notes from the team columns

In [51]:
# Importing required libraries
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# Importing cities dataset from wikipedia
cities = pd.read_html('assets/wikipedia_data.html')[1]

cities.head()

,Metropolitan area,Country,Pop.rank,Population (2016 est.)[8],B4,NFL,MLB,NBA,NHL,B6,MLS,CFL
0,New York City,United States,1,20153634,9,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3],11,Red BullsNew York City FC,—
1,Los Angeles,United States,2,13310447,8,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks,10,GalaxyLos Angeles FC[note 5],—
2,San Francisco Bay Area,United States,6,6657982,6,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7],7,Earthquakes,—
3,Chicago,United States,3,9512999,5,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks,6,Fire,—
4,Dallas–Fort Worth,United States,4,7233323,4,Cowboys,Rangers,Mavericks,Stars,5,FC Dallas,—


In [52]:
# Reducing rows and columns from cities dataset (removing last row 'totals', and keeping relevand columns)
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

# Cleaning column names
cities.columns = [x.lower().strip() for x in cities.columns]
cities.rename({'population (2016 est.)[8]': 'population'}, axis=1, inplace=True)

# Converting population into integer
cities['population'] = cities['population'].astype(int)

# Cleaning notes in team columns
leagues = cities.columns
for league in leagues:
    cities[league].replace('\[.+\]', '', regex=True, inplace=True)

cities.head()

,metropolitan area,population,nfl,mlb,nba,nhl
0,New York City,20153634,GiantsJets,YankeesMets,KnicksNets,RangersIslandersDevils
1,Los Angeles,13310447,RamsChargers,DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders,GiantsAthletics,Warriors,Sharks
3,Chicago,9512999,Bears,CubsWhite Sox,Bulls,Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


### Importing and cleaning teams dataframes

In [53]:
# Importing league teams datasets
nhl_df = pd.read_csv('assets/nhl.csv')
nfl_df = pd.read_csv('assets/nfl.csv')
nba_df = pd.read_csv('assets/nba.csv')
mlb_df = pd.read_csv('assets/mlb.csv')

nhl_df.head()

nhl_delete = nhl_df
nfl_delete = nfl_df
nba_delete = nba_df
mlb_delete = mlb_df

### Exploring & Cleaning NHL dataset. Creating functions for the rest of the tables
In this section we cleaned the NHL while in the process we created functions to clean the rest of the datasets. The cleaning process included: 
- A function to clean the column names and filter only information of year 2018
- A function to select only relevant columns for the analysis
- A function to remove notes/symbols at the end of the teams name
- A function to remove the rows that refer to divisions and not to teams
- A function to extract the cities from the teams name
- A function to generate the win-loss ratio
- A function to find the metropolitan area of each city

In the next sections these functions will be applied to the remaining dataframes

In [54]:
# DELETE:
nhl_df = nhl_delete

# Cleaning column names and filtering year function
def cols_year_cleaning(df, year = 2018):
    '''
    Function that takes a dataset and cleans column names converting them into lower case and removing any
    spaces. In addition in filters only the year desired. 
    '''
    # Cleaning column names
    df.columns = [x.strip().lower() for x in df]
    # Selecting only 2018
    df = df[df['year'] == year]
    return df

# Applying column cleaning and year filtering to nhl df
nhl_df = cols_year_cleaning(nhl_df, 2018)

# Reducing to desired columns function
def select_columns(df, cols):
    '''
    This function takes a dataframe en keeps only passed columns as a list
    '''
    df = df[cols]
    return df

nhl_df = select_columns(nhl_df, ['team', 'w', 'l'])

# Cleaning any notes or symbols at the end of the team names
def remove_notes(df, pattern):
    df['team'].replace(pattern, '', regex=True, inplace=True)
    return df

nhl_df = remove_notes(nhl_df, '\*')

# Dropping divisions rows from data
def drop_divisions(df, pattern):
    ''' 
    This function takes a dataframe and looks for a specific pattern in the 'w' column to drop those specific rows
    '''
    divisions_mask = df['w'].str.contains(pattern, regex=True) == False
    df = df[divisions_mask]
    df.reset_index(drop=True, inplace=True)
    return df

nhl_df = drop_divisions(nhl_df, 'Division')

# Separating teams from cities
def separate_cities_teams(df, twoWord_cities = ['New', 'Tampa', 'St.', 'San', 'Los']):
    df['city'] = df['team'].apply(lambda x: ' '.join(x.split()[0:2]) if x.split()[0] in twoWord_cities else x.split()[0])
    df['team'] = df['team'].apply(lambda x: ' '.join(x.split()[2:]) if x.split()[0] in twoWord_cities else ' '.join(x.split()[1:]))
    df = df[['city', 'team', 'w', 'l']]
    return df

nhl_df = separate_cities_teams(nhl_df)

# Generating win loss ratio column
def win_loss_ratio(df):
    df[['w', 'l']] = df[['w', 'l']].astype(float)
    df['ratio'] = df['w'] / df['l']
    return df

nhl_df = win_loss_ratio(nhl_df)

# Generating metropolitan area
def metropolitan_areas(df, cities, league):
    ''' 
    This function takes the league dataframe and compares the city with the metropolitan area of the cities dataframe.
    If the text inside the city is contained within the metropolitan area it assigns that metropolitam area, otherwise
    it keeps the original city and mannual inputting will be necessary.
    '''
    df['area'] = df['city']
    for area in cities['metropolitan area']:
        for city in df['area']:
            if re.search(city, area):
                df['area'].replace(city, area, inplace=True)
    
    for team_c in cities[league]:
        for team_x in df['team']:
            if re.search(team_x, team_c):
                wrong_area = df[df['team']==team_x].iloc[0]['area']
                correct_area = cities[cities[league]==team_c].iloc[0]['metropolitan area']
                if wrong_area != correct_area:
                    df['area'].replace(wrong_area, correct_area, inplace=True)
                    print('Replaced:', wrong_area, 'with:', correct_area)
                    
    return df

nhl_df = metropolitan_areas(nhl_df, cities, league='nhl')

nhl_df


Replaced: New Jersey with: New York City
Replaced: Anaheim with: Los Angeles
Replaced: San Jose with: San Francisco Bay Area
Replaced: Minnesota with: Minneapolis–Saint Paul
Replaced: Colorado with: Denver
Replaced: Florida with: Miami–Fort Lauderdale
Replaced: Arizona with: Phoenix
Replaced: Carolina with: Raleigh


,city,team,w,l,ratio,area
0,Tampa Bay,Lightning,54.0,23.0,2.347826,Tampa Bay Area
1,Boston,Bruins,50.0,20.0,2.500000,Boston
2,Toronto,Maple Leafs,49.0,26.0,1.884615,Toronto
3,Florida,Panthers,44.0,30.0,1.466667,Miami–Fort Lauderdale
4,Detroit,Red Wings,30.0,39.0,0.769231,Detroit
5,Montreal,Canadiens,29.0,40.0,0.725000,Montreal
6,Ottawa,Senators,28.0,43.0,0.651163,Ottawa
7,Buffalo,Sabres,25.0,45.0,0.555556,Buffalo
8,Washington,Capitals,49.0,26.0,1.884615,"Washington, D.C."
9,Pittsburgh,Penguins,47.0,29.0,1.620690,Pittsburgh


### Cleaning NFL dataframe

In [55]:
# DELETE
nfl_df = nfl_delete
# Cleaning column names and filtering year
nfl_df = cols_year_cleaning(nfl_df)

# Selecting columns
nfl_df = select_columns(nfl_df, ['team', 'w', 'l'])

# Removing notes
nfl_df = remove_notes(nfl_df, pattern='\*')
nfl_df = remove_notes(nfl_df, pattern='\+')

# Dropping division rows
nfl_df = drop_divisions(nfl_df, pattern='[A-Z]{3}\s')

# Separating teams and cities
nfl_df = separate_cities_teams(nfl_df, ['New', 'Tampa', 'St.', 'San', 'Los', 'Kansas', 'Green'])

# Generating win loss ratio
nfl_df = win_loss_ratio(nfl_df)

# Generating metropolitan area
nfl_df = metropolitan_areas(nfl_df, cities, league='nfl')

nfl_df

Replaced: Oakland with: San Francisco Bay Area
Replaced: New England with: Boston
Replaced: Minnesota with: Minneapolis–Saint Paul
Replaced: Arizona with: Phoenix
Replaced: Carolina with: Charlotte
Replaced: Tennessee with: Nashville


,city,team,w,l,ratio,area
0,New England,Patriots,11.0,5.0,2.200000,Boston
1,Miami,Dolphins,7.0,9.0,0.777778,Miami–Fort Lauderdale
2,Buffalo,Bills,6.0,10.0,0.600000,Buffalo
3,New York,Jets,4.0,12.0,0.333333,New York City
4,Baltimore,Ravens,10.0,6.0,1.666667,Baltimore
5,Pittsburgh,Steelers,9.0,6.0,1.500000,Pittsburgh
6,Cleveland,Browns,7.0,8.0,0.875000,Cleveland
7,Cincinnati,Bengals,6.0,10.0,0.600000,Cincinnati
8,Houston,Texans,11.0,5.0,2.200000,Houston
9,Indianapolis,Colts,10.0,6.0,1.666667,Indianapolis


In [56]:
# DELETE:
nba_df = nba_delete
# Cleaning column names and filtering year
nba_df = cols_year_cleaning(nba_df)

# Selecting columns
nba_df = select_columns(nba_df, ['team', 'w', 'l'])

# Removing notes
nba_df = remove_notes(nba_df, pattern='\*')
nba_df = remove_notes(nba_df, pattern='\(\d+\)')

# Dropping division rows
nba_df = drop_divisions(nba_df, pattern='[A-Z]{3}\s')

# Separating teams and cities
nba_df = separate_cities_teams(nba_df, ['New', 'Tampa', 'St.', 'San', 'Los', 'Kansas', 'Green', 'Golden'])

# Generating win loss ratio
nba_df = win_loss_ratio(nba_df)

# Generating metropolitan area
nba_df = metropolitan_areas(nba_df, cities, league='nba')

nba_df

Replaced: Brooklyn with: New York City
Replaced: Golden State with: San Francisco Bay Area
Replaced: Minnesota with: Minneapolis–Saint Paul
Replaced: Utah with: Salt Lake City


,city,team,w,l,ratio,area
0,Toronto,Raptors,59.0,23.0,2.565217,Toronto
1,Boston,Celtics,55.0,27.0,2.037037,Boston
2,Philadelphia,76ers,52.0,30.0,1.733333,Philadelphia
3,Cleveland,Cavaliers,50.0,32.0,1.562500,Cleveland
4,Indiana,Pacers,48.0,34.0,1.411765,Indianapolis
5,Miami,Heat,44.0,38.0,1.157895,Miami–Fort Lauderdale
6,Milwaukee,Bucks,44.0,38.0,1.157895,Milwaukee
7,Washington,Wizards,43.0,39.0,1.102564,"Washington, D.C."
8,Detroit,Pistons,39.0,43.0,0.906977,Detroit
9,Charlotte,Hornets,36.0,46.0,0.782609,Charlotte


In [57]:
# DELETE:
mlb_df = mlb_delete
# Cleaning column names and filtering year
mlb_df = cols_year_cleaning(mlb_df)

# Selecting columns
mlb_df = select_columns(mlb_df, ['team', 'w', 'l'])

# Removing notes
mlb_df = remove_notes(mlb_df, pattern='\*')
mlb_df = remove_notes(mlb_df, pattern='\(\d+\)')

# Dropping division rows

# Separating teams and cities
mlb_df = separate_cities_teams(mlb_df, ['New', 'Tampa', 'St.', 'San', 'Los', 'Kansas', 'Green', 'Golden'])

# Generating win loss ratio
mlb_df = win_loss_ratio(mlb_df)

# Generating metropolitan area
mlb_df = metropolitan_areas(mlb_df, cities, league='mlb')

mlb_df

Replaced: Oakland with: San Francisco Bay Area
Replaced: Texas with: Dallas–Fort Worth
Replaced: Minnesota with: Minneapolis–Saint Paul
Replaced: Colorado with: Denver
Replaced: Arizona with: Phoenix


,city,team,w,l,ratio,area
0,Boston,Red Sox,108.0,54.0,2.000000,Boston
1,New York,Yankees,100.0,62.0,1.612903,New York City
2,Tampa Bay,Rays,90.0,72.0,1.250000,Tampa Bay Area
3,Toronto,Blue Jays,73.0,89.0,0.820225,Toronto
4,Baltimore,Orioles,47.0,115.0,0.408696,Baltimore
5,Cleveland,Indians,91.0,71.0,1.281690,Cleveland
6,Minnesota,Twins,78.0,84.0,0.928571,Minneapolis–Saint Paul
7,Detroit,Tigers,64.0,98.0,0.653061,Detroit
8,Chicago,White Sox,62.0,100.0,0.620000,Chicago
9,Kansas City,Royals,58.0,104.0,0.557692,Kansas City


### Verifying and normalizing areas

In [58]:
# Function to find areas generated that are non-existent in the cities dataframe
def finding_missing_areas(df, cities):
    missing_areas = []
    for area in df['area']:
        if area not in list(cities['metropolitan area']):
            missing_areas.append(area)
            print('Issue:', area)
    if len(missing_areas) == 0: print('No area errors')

#### Areas double checking

Our area mapping function worked correctly. Now we have 100% coincidence in the areas between the cities and leagues datasets

In [59]:
finding_missing_areas(nhl_df, cities)
finding_missing_areas(nfl_df, cities)
finding_missing_areas(nba_df, cities)
finding_missing_areas(mlb_df, cities)

No area errors
No area errors
No area errors
No area errors


### Generating merged df by area

In [60]:
# Function that merges the dfs by area and groups values by area
def merge_areas(df):
    df = pd.merge(df, cities[['metropolitan area', 'population']], how='inner', left_on='area', right_on='metropolitan area').drop(columns='area')
    df_area = df.groupby('metropolitan area').agg({'w': (np.sum), 'l': (np.sum), 'population': (np.average)})
    df_area['win_loss_ratio'] = df_area['w'] / (df_area['w'] + df_area['l'])
    return df_area

# Creating areas dfs
areas_nhl = merge_areas(nhl_df)
areas_nfl = merge_areas(nfl_df)
areas_nba = merge_areas(nba_df)
areas_mlb = merge_areas(mlb_df)

# Array of dataframes

areas_nfl


,w,l,population,win_loss_ratio
metropolitan area,,,,
Atlanta,7.0,9.0,5789700,0.437500
Baltimore,10.0,6.0,2798886,0.625000
Boston,11.0,5.0,4794447,0.687500
Buffalo,6.0,10.0,1132804,0.375000
Charlotte,7.0,9.0,2474314,0.437500
Chicago,12.0,4.0,9512999,0.750000
Cincinnati,6.0,10.0,2165139,0.375000
Cleveland,7.0,8.0,2055612,0.466667
Dallas–Fort Worth,10.0,6.0,7233323,0.625000


### Finding correlations by league
According to these findings, the two leagues in which population and win loss ratio are correlated the highest are NBA, with a correlation of -.17, suggesting that the less populated the city the better teams perform; and MLB with a correlation of +0.15, suggesting that the more populated the city the better teams perform. 

In [61]:
corr_nhl = areas_nhl['population'].corr(areas_nhl['win_loss_ratio'])
print('Correlation nhl: {}'.format(round(corr_nhl, 5)))
corr_nfl = areas_nfl['population'].corr(areas_nfl['win_loss_ratio'])
print('Correlation nfl: {}'.format(round(corr_nfl, 5)))
corr_nba = areas_nba['population'].corr(areas_nba['win_loss_ratio'])
print('Correlation nba: {}'.format(round(corr_nba, 5)))
corr_mlb = areas_mlb['population'].corr(areas_mlb['win_loss_ratio'])
print('Correlation mlb: {}'.format(round(corr_mlb, 5)))

Correlation nhl: 0.01231
Correlation nfl: 0.00492
Correlation nba: -0.17657
Correlation mlb: 0.15052


### Finding aggregate correlation

In [93]:
cols = ['w', 'l', 'population']
# Merging datasets from all leagues
areas_tot = pd.merge(areas_nhl[cols], areas_nfl[cols], how='outer', left_index=True, right_index=True)
areas_tot.rename(columns={'w_x': 'w_nhl', 'w_y': 'w_nfl', 
                        'l_x': 'l_nhl', 'l_y': 'l_nfl',
                        'population_x': 'pop_nhl', 'population_y': 'pop_nfl'}, inplace=True)
areas_tot = areas_tot.merge(areas_nba[cols], how='outer', left_index=True, right_index=True)
areas_tot.rename(columns={'w': 'w_nba', 'l': 'l_nba', 'population': 'pop_nba'}, inplace=True)
areas_tot = areas_tot.merge(areas_mlb[cols], how='outer', left_index=True, right_index=True)
areas_tot.rename(columns={'w': 'w_mlb', 'l': 'l_mlb', 'population': 'pop_mlb'}, inplace=True)

# Adding wins and losses
def wins_add(df):
    data_w = df[['w_nhl', 'w_nfl', 'w_nba', 'w_mlb']]
    data_l = df[['l_nhl', 'l_nfl', 'l_nba', 'l_mlb']]
    data_p = df[['pop_nhl', 'pop_nfl', 'pop_nba', 'pop_mlb']]
    return pd.Series({'w': np.nansum(data_w), 'l': np.nansum(data_l), 'population': np.nanmean(data_p)})

areas_tot = areas_tot.apply(wins_add, axis='columns')

# Creating win loss ratio
areas_tot['win_loss_ratio'] = areas_tot['w'] / (areas_tot['w'] + areas_tot['l'])


In [94]:
corr_tot = areas_tot['population'].corr(areas_tot['win_loss_ratio'])
print('Correlation TOT: {}'.format(round(corr_tot, 5)))

Correlation TOT: 0.0388
